# Renewable Energy Sources - APP CSP
<i><u>Author</u> : Anthony CHUNG (25881800) - Louis GOUDERS </i>

## Context 
An agro-food industry established in the South of Spain, near Granada, would like to produce its own energy, in order to meet both its electricity and heat needs.

Hired as an energy efficiency engineer, we are asked to investigate the possibility of building a CSP plant, that should be designed to avoid injecting electricity on the grid to maximize the ROI. 

Additionally, we are also asked to think  about management of the production during the night so that the factory does not stop running, but still runs at a capacity factor of minimum 30% of the design baseload.

## Data